<a href="https://colab.research.google.com/github/greyhound101/gan_segmentation_FE/blob/main/prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [ ]:
import zipfile
with zipfile.ZipFile('/content/gdrive/My Drive/segmentation/Training_Batch1.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
pip install medpy

     |████████████████████████████████| 153kB 9.2MB/s 
     |████████████████████████████████| 47.4MB 67kB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp36-cp36m-linux_x86_64.whl size=753439 sha256=0d5be5ca9022330813f215866fe2e4ed3dc12790cff6cb7e1f2fc62ecf5ef876
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy


In [ ]:
pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 33kB/s 
     |████████████████████████████████| 368kB 51.0MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 3.2MB 43.3MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


In [ ]:
pip install keras==2.2.4

     |████████████████████████████████| 317kB 7.8MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [ ]:
!git clone https://github.com/xmengli999/H-DenseUNet

Cloning into 'H-DenseUNet'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 706 (delta 5), reused 0 (delta 0), pack-reused 697
Receiving objects: 100% (706/706), 13.77 MiB | 17.56 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [ ]:
from medpy.io import load, save
import os
import os.path
import numpy as np


def proprecessing(image_path, save_folder):

    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)
    filelist = os.listdir(image_path)
    filelist = [item for item in filelist if 'volume' in item]
    for file in filelist:
        img, img_header = load(image_path+file)
        img[img < -200] = -200
        img[img > 250] = 250
        img = np.array(img, dtype='float32')
        print ("Saving image "+file)
        save(img, "./data/" + save_folder + file)

def generate_livertxt(image_path, save_folder):
    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("data/"+save_folder+'LiverPixels'):
        os.mkdir("data/"+save_folder+'LiverPixels')


    #change
    #131 to 5

    for i in range(0,131):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open('data/' +save_folder+'/LiverPixels/liver_' + str(i) + '.txt', 'w')
        index = np.where(livertumor==1)
        x = index[0]
        y = index[1]
        z = index[2]
        np.savetxt(f, np.c_[x,y,z], fmt="%d")
	
        f.write("\n")
        f.close()

def generate_tumortxt(image_path, save_folder):
    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("data/"+save_folder+'TumorPixels'):
        os.mkdir("data/"+save_folder+'TumorPixels')

    for i in range(0,131):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open("data/"+save_folder+"/TumorPixels/tumor_"+str(i)+'.txt','w')
        index = np.where(livertumor==2)

        x = index[0]
        y = index[1]
        z = index[2]

        np.savetxt(f,np.c_[x,y,z],fmt="%d")

        f.write("\n")
        f.close()

def generate_txt(image_path, save_folder):
    if not os.path.exists("data/"+save_folder):
        os.mkdir("data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("data/"+save_folder+'LiverBox'):
        os.mkdir("data/"+save_folder+'LiverBox')
    for i in range(0,131):
        values = np.loadtxt('data/myTrainingDataTxt/LiverPixels/liver_' + str(i) + '.txt', delimiter=' ', usecols=[0, 1, 2])
        a = np.min(values, axis=0)
        b = np.max(values, axis=0)
        box = np.append(a,b, axis=0)
        np.savetxt('data/myTrainingDataTxt/LiverBox/box_'+str(i)+'.txt', box,fmt='%d')

#change
#create data
#change training data path

proprecessing(image_path='/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/', save_folder='myTrainingData/')


#commenting test
# proprecessing(image_path='data/TestData/', save_folder='myTestData/')


print ("Generate liver txt ")
generate_livertxt(image_path='/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/', save_folder='myTrainingDataTxt/')
print ("Generate tumor txt")
generate_tumortxt(image_path='/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/', save_folder='myTrainingDataTxt/')
print ("Generate liver box ")
generate_txt(image_path='data/TrainingData/', save_folder='myTrainingDataTxt/')


Saving image volume-4.nii
Saving image volume-0.nii
Saving image volume-1.nii
Saving image volume-3.nii
Saving image volume-2.nii
Generate liver txt 
Generate tumor txt
Generate liver box 


In [ ]:
#change
#keras -2.0.8

from __future__ import print_function
# import sys
# sys.path.insert(0,'Keras-2.0.8')
from multiprocessing.dummy import Pool as ThreadPool
import random
from medpy.io import load
import numpy as np
import argparse
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import keras.backend as K
# from loss import weighted_crossentropy_2ddense
import os
# from keras.utils2.multi_gpu import make_parallel
# from denseunet import DenseUNet
from skimage.transform import resize
K.set_image_dim_ordering('tf')

#  global parameters
# parser = argparse.ArgumentParser(description='Keras 2d denseunet Training')
# #  data folder
# parser.add_argument('-data', type=str, default='data/', help='test images')
# parser.add_argument('-save_path', type=str, default='Experiments/')
# #  other paras
# parser.add_argument('-b', type=int, default=40)
# parser.add_argument('-input_size', type=int, default=224)
# parser.add_argument('-model_weight', type=str, default='./model/densenet161_weights_tf.h5')
# parser.add_argument('-input_cols', type=int, default=3)

#  data augment
# parser.add_argument('-mean', type=int, default=48)
# parser.add_argument('-thread_num', type=int, default=14)
# args = parser.parse_args()

MEAN = 48
thread_num = 14
input_size=224
b=1
input_cols=3

liverlist = [32,34,38,41,47,87,89,91,105,106,114,115,119]
trainidx = list(range(5))
img_list = []
tumor_list = []
minindex_list = []
maxindex_list = []
tumorlines = []
tumoridx = []
liveridx = []
liverlines = []

for idx in range(196):
        img, img_header = load('/content/data/myTrainingData/volume-'+str(idx)+'.nii')
        tumor, tumor_header = load('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/segmentation-'+str(idx)+'.nii')
        img_list.append(img)
        tumor_list.append(tumor)

        maxmin = np.loadtxt('/content/data/myTrainingDataTxt/LiverBox/box_'+str(idx)+'.txt')
        minindex = maxmin[0:3]
        maxindex = maxmin[3:6]
        minindex = np.array(minindex, dtype='int')
        maxindex = np.array(maxindex, dtype='int')
        minindex[0] = max(minindex[0] - 3, 0)
        minindex[1] = max(minindex[1] - 3, 0)
        minindex[2] = max(minindex[2] - 3, 0)
        maxindex[0] = min(img.shape[0], maxindex[0] + 3)
        maxindex[1] = min(img.shape[1], maxindex[1] + 3)
        maxindex[2] = min(img.shape[2], maxindex[2] + 3)
        minindex_list.append(minindex)
        maxindex_list.append(maxindex)
        f1 = open('/content/data/myTrainingDataTxt/TumorPixels/tumor_'+str(idx)+'.txt')
        tumorline = f1.readlines()
        tumorlines.append(tumorline)
        tumoridx.append(len(tumorline))
        f1.close()
        f2 = open('/content/data/myTrainingDataTxt/LiverPixels/liver_'+str(idx)+'.txt')
        liverline = f2.readlines()
        liverlines.append(liverline)
        liveridx.append(len(liverline))
        f2.close()
        np.save('/content/gdrive/MyDrive/'+str(idx)+'.npy',[trainidx, img_list, tumor_list, tumorlines, liverlines, tumoridx, liveridx, minindex_list, maxindex_list])


------------------------------
Creating and compiling model...
------------------------------
------------------------------
Fitting model......
------------------------------


/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
5/5 [==============================] - 136s 27s/step - loss: 1.8731
Epoch 2/5
5/5 [==============================] - 22s 4s/step - loss: 1.4738
Epoch 3/5
5/5 [==============================] - 22s 4s/step - loss: 0.9403
Epoch 4/5
5/5 [==============================] - 22s 4s/step - loss: 0.7511
Epoch 5/5
5/5 [==============================] - 23s 5s/step - loss: 1.0408
Finised Training .......
